# PyUAT: Tracking example with custom age model

In this example, we utilize the modularity of PyUAT and add a custom age model for the cells. We demonstrate this again at the example of a time-lapse sequence of the Tracking-One-In-A-Million dataset:

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.7260137.svg)](https://doi.org/10.5281/zenodo.7260137)

## 1. Install software dependencies

Install the required software packages

In [ ]:
%pip install gurobipy moviepy
%pip install uatrack==0.0.3

## 2. Download image data and segmentation

We download the imaging data and the ground truth segmentation & tracking data 🚀

In [ ]:
!wget -O filtered_0.json https://fz-juelich.sciebo.de/s/5vBB6tW8c2DpaU3/download
!wget -O 00_stack.tif https://fz-juelich.sciebo.de/s/Xge7fj56QM5ev7q/download

# 3. Prepare the tracking configuration

In order to track cell using `PyUAT`, we need to define models that score the properties of assignments. That is performed in the section below and results in the construction of assignment generators that generate and score the different assignment types.

In [1]:
subsampling_factor = 2
end_frame = 400

# Load segmentation data

In [2]:
# TODO load data

from uatrack.utils import load_data

image_file = "00_stack.tif"
tracking_file = "filtered_0.json"

overlay, tracking_graph = load_data(tracking_file, subsampling_factor=subsampling_factor, end_frame=end_frame)

/home/johannes/miniconda3/envs/uat-examples/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-10 15:40:00,154	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
findfont: Font family ['DejaVuSans'] not found. Falling back to DejaVu Sans.


In [3]:
print("Total number of cell detections: ", len(overlay))

Total number of cell detections:  3360


## 4. Perform the tracking

Here, we first defnie

In [10]:
import time

from uatrack.utils import extract_single_cell_information, save_tracking
from uatrack.core import simpleTracking

from uatrack.config import make_assignmet_generators, use_first_order_model, split_dist_computer, migration_prob, create_constant_disappear_model, log_sum, use_nearest_neighbor
from uatrack.models import ModelExecutor
from scipy.stats import binom 

import numpy as np
from functools import partial

input_file = "segmentation.json"
output_file = "simpleTracking.json.gz"

# Fix arguments
num_particles = 1 
num_cores = 1
max_num_hypotheses = 1
cutOff = -1
max_num_solutions = 1

print("Extract single-cell information...")
df, all_detections = extract_single_cell_information(overlay)

print("Setup assignment generators...")
# arrange single-cell information into numpy arrays (greatly increases the speed, as data can be immediately indexed)
data = {
    "area": np.array(df["area"].to_list(), dtype=np.float32),
    "centroid": np.array(df["centroid"].to_list(), dtype=np.float32),
    "major_extents": np.array(df["major_extents"].to_list(), dtype=np.float32),
    "major_axis": np.array(df["major_axis"].to_list(), dtype=np.float32),
}

# create biologically motivated models
(
    constant_new_models,
    constant_end_models,
    migration_models,
    split_models,
) = use_nearest_neighbor(data=data, subsampling=subsampling_factor)


constant_new_model, constant_end_model = create_constant_disappear_model(1e-3)
constant_new_models = [constant_new_model]
constant_end_models = [constant_end_model]


####################
### Custom Model ###
####################

def division_prob(x):
    diff = np.abs(x-60)
    # use a binomial distribution with mean 60 and compute probability based on differences
    probs = binom.cdf(60-diff, n=120, p=0.5) + binom.sf(60+diff, n=120, p=0.5)
    probs[x == -1] = 0.5
    return probs

# add migration model as 1-division_prob
migration_models.append(
    ModelExecutor(
        quantity_computer=split_dist_computer,
        model=lambda x: np.log(1-division_prob(x)),
        df=data
    )
)

# add division model
split_models.append(
    ModelExecutor(
        quantity_computer=split_dist_computer,
        model=lambda x: np.log(division_prob(x)),
        df=data
    )
)

# create the assignment candidate generators
assignment_generators = make_assignmet_generators(
    df=df,
    data=data,
    constant_new_models=constant_new_models,
    constant_end_models=constant_end_models,
    migration_models=migration_models,
    split_models=split_models,
)

print("Perform tracking...")
# start tracking
start = time.time()
res = simpleTracking(
    df,
    assignment_generators,
    num_particles,
    num_cores=num_cores,
    max_num_hypotheses=max_num_hypotheses,
    cutOff=cutOff,
    max_num_solutions=max_num_solutions,
    mip_method="CBC",  # use "GRB" if you have gurobi installed
)
end = time.time()

print("time for tracking", end - start)

save_tracking(res[0], all_detections, output_file)


Extract single-cell information...
Setup assignment generators...
Perform tracking...


Perform tracking:  48%|████▊     | 96/200 [00:04<00:08, 11.78it/s]WARNING:py.warnings:/tmp/ipykernel_562160/292980591.py:64: RuntimeWarning: divide by zero encountered in log
  model=lambda x: np.log(1-division_prob(x)),

Perform tracking: 100%|██████████| 200/200 [01:15<00:00,  2.65it/s]


time for tracking 75.49974298477173


## 4. Render tracking video

In order to visually assess the quality of the tracking, we can render it into a video. We use the `acia` library for that.

In [11]:
from acia.tracking.formats import parse_simple_tracking
import gzip

# read the tracking result
with gzip.open("simpleTracking.json.gz") as input_file:
  overlay, pred_tracking_graph = parse_simple_tracking(input_file.read())

In [12]:
import tifffile
import numpy as np

from acia.segm.local import InMemorySequenceSource
from acia.tracking import subsample_tracking, TrackingSourceInMemory
from acia.viz import render_tracking, render_segmentation, render_video


ts = TrackingSourceInMemory(overlay, pred_tracking_graph)

image_stack = tifffile.imread(image_file)
raw_images = InMemorySequenceSource(np.repeat(image_stack[:end_frame][::subsampling_factor,...,None], repeats=3, axis=-1))

segm_images = render_segmentation(raw_images, overlay)
tracked_images = render_tracking(segm_images, overlay, pred_tracking_graph)

render_video(tracked_images, "tracked.mp4", 10, "vp9")

200it [00:00, 205.28it/s]
200it [00:00, 230.30it/s]


Moviepy - Building video /home/johannes/projects/work/uat-new/tracked.mp4.
Moviepy - Writing video /home/johannes/projects/work/uat-new/tracked.mp4



Moviepy - Done !
Moviepy - video ready /home/johannes/projects/work/uat-new/tracked.mp4


# Show the video

In [13]:
# embed the video in the notebook
import moviepy.editor
moviepy.editor.ipython_display("tracked.mp4", maxduration=300)